<a href="https://colab.research.google.com/github/vivek0464/A-novel-image-encryption-scheme/blob/main/Image_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
import numpy as np
import matplotlib.image as img
from google.colab import files
from PIL import Image

In [ ]:
uploaded = files.upload()
imgP = img.imread('image.tiff')

Saving image.tiff to image (3).tiff


In [ ]:
def MeanFract_list(L):
    sum = 0
    for i in L:
        sum = sum+i
    return math.modf(sum)[0]

def MeanFract_numpy(M, r):
    sum = 0
    for i in range(M.shape[0]):
        if i == r: 
            continue
        for j in M[i]:
            sum = sum+j
    return math.modf(sum)[0]

def generateSeq(x0, a, b, len):
    seq = list()
    seq.append(x0)
    for i in range(len-1):
        x = math.cos(a/(x0**b))
        x0 = x
        seq.append(x0)
    return seq

In [ ]:
def generateKey(T):
    key = list()
    S = generateSeq(random.uniform(0.9,1), random.uniform(0.9,1), random.randint(1,20), T*8)
    k = 0
    for i in range(T):
        L = list()
        L.append(random.randint(1, 50))
        for j in range(4):
            L.append(S[k])
            k = k+1
        L.append(random.randint(1, 50))
        for j in range(4):
            L.append(S[k])
            k = k+1
        key.append(L)
    return key

In [ ]:
def encrypt(imgP, key, T):
    C = np.ndarray(imgP.shape)

    for color in range(3):
        P = imgP[:,:,color].copy()

        for r in range(T):
            for k in range(1, 3):
                M = P.shape[0]
                x0 = key[r][5*(k-1)+1] + MeanFract_list(key[r])
                a = key[r][5*(k-1)+4]
                b = key[r][5]

                # Generate a random-like path using 1-DCF map
                RLP = generateSeq(x0, a, b, M)

                # Normalize the RLP sequence
                RLP = [i[0] for i in sorted(enumerate(RLP), key=lambda x:x[1])]
                RLP = np.array(RLP)
                
                avg = MeanFract_numpy(P, RLP[0])

                # Generate two sequences S1 and S2 by 1-DCF map
                x0 = key[r][5*(k-1)+2] + avg
                S1 = generateSeq(x0, a, b, M)
                x0 = key[r][5*(k-1)+3]
                S2 = generateSeq(x0, a, b, M)

                # Normalize S1 and S2
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S1 = np.array(S1)
                S2 = np.array(S2)

                P[RLP[0]] = P[RLP[0]] ^ S1
                for i in range(1, M):
                    P[RLP[i]] = (P[RLP[i]] + (P[RLP[i-1]] ^ S2)) % 256

                avg = MeanFract_numpy(P, RLP[M-1])

                # Generate two sequences S1 and S2 by 1-DCF map
                x0 = key[r][5*(k-1)+3] + avg
                S1 = generateSeq(x0, a, b, M)
                x0 = key[r][5*(k-1)+2]
                S2 = generateSeq(x0, a, b, M)

                # Normalize S1 and S2
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S1 = np.array(S1)
                S2 = np.array(S2)

                P[RLP[M-1]] = P[RLP[M-1]] ^ S1
                for i in range(M-2, -1, -1):
                    P[RLP[i]] = (P[RLP[i]] + (P[RLP[i+1]] ^ S2)) % 256

                P = P.transpose()

        C[:,:,color] = P

    C = C.astype(np.uint8)
    return C

In [ ]:
def decrypt(imgP, key, T):
    C = np.ndarray(imgP.shape)

    for color in range(3):
        P = imgP[:,:,color].copy()

        for r in range(T-1, -1, -1):
            for k in range(2, 0, -1):
                M = P.shape[0]
                x0 = key[r][5*(k-1)+1] + MeanFract_list(key[r])
                a = key[r][5*(k-1)+4]
                b = key[r][5]

                RLP = generateSeq(x0, a, b, M)
                RLP = [i[0] for i in sorted(enumerate(RLP), key=lambda x:x[1])]
                RLP = np.array(RLP)

                P = P.transpose()

                x0 = key[r][5*(k-1)+2]
                S2 = generateSeq(x0, a, b, M)
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S2 = np.array(S2)

                for i in range(M-1):
                    P[RLP[i]] = (P[RLP[i]] - (P[RLP[i+1]] ^ S2) + 256) % 256
                
                avg = MeanFract_numpy(P, RLP[M-1])
                x0 = key[r][5*(k-1)+3] + avg
                S1 = generateSeq(x0, a, b, M)
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S1 = np.array(S1)

                P[RLP[M-1]] = P[RLP[M-1]] ^ S1

                x0 = key[r][5*(k-1)+3]
                S2 = generateSeq(x0, a, b, M)
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S2 = np.array(S2)

                for i in range(M-1, 0, -1):
                    P[RLP[i]] = (P[RLP[i]] - (P[RLP[i-1]] ^ S2) + 256) % 256
                
                avg = MeanFract_numpy(P, RLP[0])
                x0 = key[r][5*(k-1)+2] + avg
                S1 = generateSeq(x0, a, b, M)
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S1 = np.array(S1)

                P[RLP[0]] = P[RLP[0]] ^ S1

        C[:,:,color] = P

    C = C.astype(np.uint8)
    return C

In [ ]:
def shuffling(imgP, key):
    C = np.ndarray(imgP.shape)

    for color in range(3):
        P = imgP[:,:,color].copy()
        M = P.shape[0]
        N = P.shape[1]
        
        x0 = key[0][1]
        a = key[0][2]
        b = key[0][0]
        
        S = generateSeq(x0, a, b, M*N)
        S = [i[0] for i in sorted(enumerate(S), key=lambda x:x[1])]

        G = list()
        for i in range(M):
            L = list()
            for j in range(N):
                L.append(P[S[i*N+j]//N][S[i*N+j]%N])
            G.append(L)
        
        G = np.array(G)
        C[:,:,color] = G
    
    C = C.astype(np.uint8)
    return C

In [ ]:
def deshuffling(imgP, key):
    C = np.ndarray(imgP.shape)

    for color in range(3):
        P = imgP[:,:,color].copy()
        M = P.shape[0]
        N = P.shape[1]
        
        x0 = key[0][1]
        a = key[0][2]
        b = key[0][0]
        
        S = generateSeq(x0, a, b, M*N)
        S = [i[0] for i in sorted(enumerate(S), key=lambda x:x[1])]

        G = [[0 for i in range(N)] for j in range(M)]
        for i in range(M):
            for j in range(N):
                G[S[i*N+j]//N][S[i*N+j]%N] = P[i][j]
        
        G = np.array(G)
        C[:,:,color] = G
    
    C = C.astype(np.uint8)
    return C

In [ ]:
def encrypt_(imgP, key, T):
    # compute average
    avg = 0
    for i in range(imgP.shape[0]):
        for j in range(imgP.shape[1]):
            for k in range(3):
                avg = avg+imgP[i][j][k]
    avg = avg / (imgP.shape[0]*imgP.shape[1]*3)

    # Shuffling
    shuffled = shuffling(imgP, key)

    C = np.ndarray(imgP.shape)

    for color in range(3):
        P = shuffled[:,:,color].copy()
        P = P.astype(int)

        for r in range(T):
            for k in range(1, 3):
                M = P.shape[0]
                x0 = key[r][5*(k-1)+1] + MeanFract_list(key[r])
                a = key[r][5*(k-1)+4]
                b = key[r][5]

                # Generate a random-like path using 1-DCF map
                RLP = generateSeq(x0, a, b, M)

                # Normalize the RLP sequence
                RLP = [i[0] for i in sorted(enumerate(RLP), key=lambda x:x[1])]
                RLP = np.array(RLP)

                # Generate two sequences S1 and S2 by 1-DCF map
                x0 = key[r][5*(k-1)+2] + avg
                S1 = generateSeq(x0, a, b, M)
                x0 = key[r][5*(k-1)+3]
                S2 = generateSeq(x0, a, b, M)

                # Normalize S1 and S2
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S1 = np.array(S1)
                S2 = np.array(S2)

                P[RLP[0]] = P[RLP[0]] ^ S1
                for i in range(1, M):
                    P[RLP[i]] = (P[RLP[i]] + (P[RLP[i-1]] ^ S2)) % 256

                # Generate two sequences S1 and S2 by 1-DCF map
                x0 = key[r][5*(k-1)+3] + avg
                S1 = generateSeq(x0, a, b, M)
                x0 = key[r][5*(k-1)+2]
                S2 = generateSeq(x0, a, b, M)

                # Normalize S1 and S2
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S1 = np.array(S1)
                S2 = np.array(S2)

                P[RLP[M-1]] = P[RLP[M-1]] ^ S1
                for i in range(M-2, -1, -1):
                    P[RLP[i]] = (P[RLP[i]] + (P[RLP[i+1]] ^ S2)) % 256

        C[:,:,color] = P

    C = C.astype(np.uint8)
    return C, avg

In [ ]:
def decrypt_(imgP, key, T, avg):
    C = np.ndarray(imgP.shape)

    for color in range(3):
        P = imgP[:,:,color].copy()
        P = P.astype(int)

        for r in range(T-1, -1, -1):
            for k in range(2, 0, -1):
                M = P.shape[0]
                x0 = key[r][5*(k-1)+1] + MeanFract_list(key[r])
                a = key[r][5*(k-1)+4]
                b = key[r][5]

                RLP = generateSeq(x0, a, b, M)
                RLP = [i[0] for i in sorted(enumerate(RLP), key=lambda x:x[1])]
                RLP = np.array(RLP)

                x0 = key[r][5*(k-1)+2]
                S2 = generateSeq(x0, a, b, M)
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S2 = np.array(S2)

                for i in range(M-1):
                    P[RLP[i]] = (P[RLP[i]] - (P[RLP[i+1]] ^ S2) + 256) % 256
                
                x0 = key[r][5*(k-1)+3] + avg
                S1 = generateSeq(x0, a, b, M)
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S1 = np.array(S1)

                P[RLP[M-1]] = P[RLP[M-1]] ^ S1

                x0 = key[r][5*(k-1)+3]
                S2 = generateSeq(x0, a, b, M)
                S2 = [int(math.modf(i*1e7)[1] % 256) for i in S2]
                S2 = np.array(S2)

                for i in range(M-1, 0, -1):
                    P[RLP[i]] = (P[RLP[i]] - (P[RLP[i-1]] ^ S2) + 256) % 256
                
                x0 = key[r][5*(k-1)+2] + avg
                S1 = generateSeq(x0, a, b, M)
                S1 = [int(math.modf(i*1e7)[1] % 256) for i in S1]
                S1 = np.array(S1)

                P[RLP[0]] = P[RLP[0]] ^ S1

        C[:,:,color] = P

    C = deshuffling(C, key)
    C = C.astype(np.uint8)
    return C

In [ ]:
T = 20
key = generateKey(T)

In [ ]:
imgC, avg = encrypt_(imgP, key, T)

for i in range(256):
    for j in range(148, 168):
        imgC[i][j] = np.array([0, 0, 0])

# for i in range(70):
#     for j in range(70):
#         imgC[i][j] = np.array([0, 0, 0])

imgC = imgC.astype(np.uint8)
img = Image.fromarray(imgC, 'RGB')
img.save('Cipher.png')

imgP_ = decrypt_(imgC, key, T, avg)
img = Image.fromarray(imgP_, 'RGB')
img.save('Decrypted.png')

In [ ]:
imgC = encrypt(imgP, key, T)
imgC = imgC.astype(np.uint8)
img = Image.fromarray(imgC, 'RGB')
img.save('Cipher.png')

imgP_ = decrypt(imgC, key, T)
img = Image.fromarray(imgP_, 'RGB')
img.save('Decrypted.png')